In [1]:
import os
import time
import numpy as np
import pandas as pd

In [2]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, ShuffleSplit, RandomizedSearchCV
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn import linear_model
from sklearn import svm
from xgboost.sklearn import XGBRegressor

In [3]:
def rmsle(h, y):
    return np.sqrt(np.square(np.log(h + 1) - np.log(y + 1)).mean())
rmsle_scorer = metrics.make_scorer(rmsle, greater_is_better=False)

In [4]:
df_train = pd.read_csv("../data/train_additional.csv")
df_test = pd.read_csv("../data/test_additional.csv")

In [5]:
features = list(set(df_train.columns.tolist()) - set(['formation_energy_ev_natom',
                                                'bandgap_energy_ev', 
                                                'id',
                                                'lattice_angle_alpha_degree_r',
                                                'lattice_angle_beta_degree_r',
                                                'lattice_angle_alpha_degree_r']))
targets = ['formation_energy_ev_natom', 'bandgap_energy_ev']

y_e = df_train['formation_energy_ev_natom']
y_be = df_train['bandgap_energy_ev']

X = df_train[features]

In [6]:
dist_features = [x for x in df_train.columns.tolist() if x.startswith('dist')]
path_features = [x for x in df_train.columns.tolist() if x.startswith('path')]
angle_features = [x for x in df_train.columns.tolist() if x.startswith('angle')]
dihedral_features = [x for x in df_train.columns.tolist() if x.startswith('dihedral')]
percent_features = [x for x in df_train.columns.tolist() if x.startswith('percent')]
avg_features = [x for x in df_train.columns.tolist() if x.startswith('avg')]
lattice_angle_features = [x for x in df_train.columns.tolist() if x.startswith('lattice_angle')]

In [7]:
feature_engineering =['avg_IP',
                     'avg_rd_max',
                     'avg_LUMO',
                     'avg_electronegativity',
                     'avg_rp_max',
                     'percent_atom_in',
                     'percent_atom_al',
                     'percent_atom_ga',
                     'dist_23',
                     'dist_25',
                     'dist_16',
                     'dist_46',
                     'dist_15',
                     'dist_40',
                     'dist_33',
                     'dist_29',
                     'dist_18',
                     'dist_8',
                     'dist_77',
                     'dist_65',
                     'dist_47',
                     'dist_20',
                     'dist_30',
                     'dist_49',
                     'path_30',
                     'path_20',
                     'path_10',
                     'path_0',
                     'path_40',
                     'path_49',
                     'angle_179',
                     'angle_86',
                     'angle_169',
                     'angle_171',
                     'angle_95',
                     'angle_93',
                     'angle_121',
                     'angle_89',
                     'angle_108',
                     'angle_91',
                     'angle_165',
                     'dihedral_75',
                     'dihedral_154',
                     'dihedral_66',
                     'dihedral_7',
                     'dihedral_178',
                     'dihedral_97',
                     'dihedral_85',
                     'dihedral_131',
                     'dihedral_24',
                     'dihedral_91',
                     'dihedral_135',
                     'dihedral_79',
                     'dihedral_177',
                     'dihedral_119',
                     'dihedral_2',
                     'A_8',
                     'A_5',
                     'A_3',
                     'A_2',
                     'A_0',
                     'A_7',
                     'A_6',
                     'A_1',
                     'A_4',
                     'B_8',
                     'B_6',
                     'B_5',
                     'B_2',
                     'B_3',
                     'B_1',
                     'B_0',
                     'B_7',
                     'B_4',
                     'R_1',
                     'R_2',
                     'R_0',
                     'r_2',
                     'r_1',
                     'r_0']

In [8]:
# when labels are in [0,1] we can also use reg:logistic
# Create a parameter grid to search for best parameters for everything in the pipeline
param_grid = {
                #'min_child_weight': [0.01,0.05,0.1, 0.5,1,1.5,2], # due to high class imbalance
                'booster':['gbtree'],
                'objective': ['reg:linear'],
                'max_depth': [1,2,3,4,5,6,7],
                'nthread': [12],
                #'max_delta_step': [1.8],
                #'colsample_bytree': [0.4],
                'subsample': np.linspace(0.7,0.99, 100),
                'learning_rate': np.linspace(0.01,0.3, 1000), #0.025
                'gamma': np.linspace(0.01,0.3, 1000),
                'n_estimators' : [250],
                #'tree_method': ['gpu_hist'],
                'silent': [1],
                'missing':[-999],
                'n_jobs': [12],
                #'updater': 'grow_gpu',
                #'scale_pos_weight' : [0.5,1,1.5] # due to high class imbalance
              }

# Normalized Gini Scorer
rmsle_scorer = metrics.make_scorer(rmsle, greater_is_better=False)

# Initialize Grid Search Model
model = RandomizedSearchCV(estimator  = XGBRegressor(),
                           param_distributions = param_grid,
                           n_iter = 250,
                           #param_grid = param_grid,
                           scoring    = rmsle_scorer,
                           verbose    = 2,
                           n_jobs     = 12,
                           iid        = True,
                           refit      = True,
                           cv         = ShuffleSplit(n_splits=5, test_size=.2))
# Fit Grid Search Model
X_gbx = df_train[['lattice_angle_alpha_degree',
                 'lattice_angle_gamma_degree',
                 'number_of_total_atoms',
                 'vol',
                 'atomic_density',
                 'lattice_vector_1_ang',
                 'lattice_vector_2_ang',
                 'lattice_vector_3_ang']
                 +feature_engineering].values
y_gbx = np.log(y_e.values+1)


model.fit(X_gbx, y_gbx)
print("Best score: %0.3f" % model.best_score_)
print("Best parameters set:")
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

# Get best model
best_model = model.best_estimator_

# Fit model with best parameters optimized for normalized_gini
#best_model.fit(train,labels)
print('this took {}-seconds'.format(time.time()-t))

Fitting 5 folds for each of 250 candidates, totalling 1250 fits
[CV] subsample=0.884545454545, silent=1, objective=reg:linear, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=4, learning_rate=0.203623623624, gamma=0.0428028028028, booster=gbtree 
[CV] subsample=0.884545454545, silent=1, objective=reg:linear, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=4, learning_rate=0.203623623624, gamma=0.0428028028028, booster=gbtree 
[CV] subsample=0.884545454545, silent=1, objective=reg:linear, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=4, learning_rate=0.203623623624, gamma=0.0428028028028, booster=gbtree 
[CV] subsample=0.884545454545, silent=1, objective=reg:linear, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=4, learning_rate=0.203623623624, gamma=0.0428028028028, booster=gbtree 
[CV] subsample=0.884545454545, silent=1, objective=reg:linear, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=4, learn

[CV]  subsample=0.86696969697, silent=1, objective=reg:linear, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=6, learning_rate=0.296226226226, gamma=0.102022022022, booster=gbtree, total= 7.7min
[CV] subsample=0.925555555556, silent=1, objective=reg:linear, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=6, learning_rate=0.156886886887, gamma=0.173433433433, booster=gbtree 
[CV]  subsample=0.711717171717, silent=1, objective=reg:linear, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=2, learning_rate=0.0323523523524, gamma=0.120890890891, booster=gbtree, total= 1.6min
[CV] subsample=0.925555555556, silent=1, objective=reg:linear, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=6, learning_rate=0.156886886887, gamma=0.173433433433, booster=gbtree 
[CV]  subsample=0.86696969697, silent=1, objective=reg:linear, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=6, learning_rate=0.296226226226, gamma=0.10202

[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:  7.9min


[CV]  subsample=0.711717171717, silent=1, objective=reg:linear, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=2, learning_rate=0.0323523523524, gamma=0.120890890891, booster=gbtree, total= 1.6min
[CV] subsample=0.925555555556, silent=1, objective=reg:linear, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=6, learning_rate=0.156886886887, gamma=0.173433433433, booster=gbtree 
[CV]  subsample=0.711717171717, silent=1, objective=reg:linear, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=2, learning_rate=0.0323523523524, gamma=0.120890890891, booster=gbtree, total= 1.6min
[CV] subsample=0.802525252525, silent=1, objective=reg:linear, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=3, learning_rate=0.102312312312, gamma=0.155145145145, booster=gbtree 
[CV]  subsample=0.711717171717, silent=1, objective=reg:linear, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=2, learning_rate=0.0323523523524, gamma=0.1

[CV]  subsample=0.94898989899, silent=1, objective=reg:linear, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=3, learning_rate=0.0587687687688, gamma=0.241941941942, booster=gbtree, total= 2.5min
[CV] subsample=0.752727272727, silent=1, objective=reg:linear, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=4, learning_rate=0.282002002002, gamma=0.262262262262, booster=gbtree 
[CV]  subsample=0.925555555556, silent=1, objective=reg:linear, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=6, learning_rate=0.156886886887, gamma=0.173433433433, booster=gbtree, total= 8.0min
[CV] subsample=0.984141414141, silent=1, objective=reg:linear, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=7, learning_rate=0.105505505506, gamma=0.260520520521, booster=gbtree 
[CV]  subsample=0.94898989899, silent=1, objective=reg:linear, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=3, learning_rate=0.0587687687688, gamma=0.2419

...

[CV] subsample=0.846464646465, silent=1, objective=reg:linear, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=3, learning_rate=0.288388388388, gamma=0.248618618619, booster=gbtree 
[CV]  subsample=0.708787878788, silent=1, objective=reg:linear, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=4, learning_rate=0.0767667667668, gamma=0.039029029029, booster=gbtree, total= 3.7min
[CV] subsample=0.846464646465, silent=1, objective=reg:linear, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=3, learning_rate=0.288388388388, gamma=0.248618618619, booster=gbtree 
[CV]  subsample=0.708787878788, silent=1, objective=reg:linear, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=4, learning_rate=0.0767667667668, gamma=0.039029029029, booster=gbtree, total= 3.7min
[CV] subsample=0.846464646465, silent=1, objective=reg:linear, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=3, learning_rate=0.288388388388, gamma=0.248

[CV] subsample=0.910909090909, silent=1, objective=reg:linear, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=5, learning_rate=0.0520920920921, gamma=0.152822822823, booster=gbtree 
[CV]  subsample=0.820101010101, silent=1, objective=reg:linear, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=5, learning_rate=0.0718318318318, gamma=0.298548548549, booster=gbtree, total= 5.6min
[CV] subsample=0.910909090909, silent=1, objective=reg:linear, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=5, learning_rate=0.0520920920921, gamma=0.152822822823, booster=gbtree 
[CV]  subsample=0.931414141414, silent=1, objective=reg:linear, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=6, learning_rate=0.228008008008, gamma=0.208268268268, booster=gbtree, total= 8.4min
[CV] subsample=0.910909090909, silent=1, objective=reg:linear, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=5, learning_rate=0.0520920920921, gamma=0.1

[Parallel(n_jobs=12)]: Done 1250 out of 1250 | elapsed: 482.2min finished


Best score: -0.028
Best parameters set:
	booster: 'gbtree'
	gamma: 0.012902902902902903
	learning_rate: 0.18678678678678676
	max_depth: 3
	missing: -999
	n_estimators: 250
	n_jobs: 12
	nthread: 12
	objective: 'reg:linear'
	silent: 1
	subsample: 0.86989898989898984


NameError: name 't' is not defined

In [ ]:
print("Best score: %0.3f" % model.best_score_)
print("Best parameters set:")
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

# Get best model
best_model = model.best_estimator_

In [ ]:
(0.045+0.027)/2